In [1]:
import torch

In [2]:
shape = (16, 3, 64, 64)

x_t = torch.zeros(shape)
x_L = torch.zeros(shape)

In [5]:
x_test = torch.concat((x_t, x_L), dim=1)
x_test.shape

torch.Size([16, 6, 64, 64])

In [9]:
x_test[:4].shape

torch.Size([4, 6, 64, 64])